In [45]:
# loading the API keys in environment variables
import os, json
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.tools import tool

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [46]:
# If you have langsmith access, run the following to trace and debuy your agent

from langsmith import Client
client = Client()

os.environ['LANGCHAIN_PROJECT'] = "contoso-agent"

In [47]:
openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

chat_model = os.environ['GPT4_MODEL_NAME']
embedding_model = os.environ['EMBEDDING_MODEL_NAME']
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(model=embedding_model, chunk_size=1)
vector_store_endpoint: str = os.environ['AZURE_COGNITIVE_SEARCH_ENDPOINT']
vector_store_password: str = os.environ['AZURE_COGNITIVE_SEARCH_KEY']
index_name: str = "contoso-coffee-index"

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_endpoint,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [ ]:
# from pydantic import BaseModel, Field
# from langchain.tools import tool, StructuredTool

# # Define the input schema
# class QueryInput(BaseModel):
#     question: str = Field(description="Question inputted by the user to retrieve the top documents from the Crypto vector store or knowledge base")
#     k: int = Field(default=10, description="The number of top documents to retrieve")

    
# @tool(args_schema=QueryInput)  
# def retrieve(question:str, k:int) -> str:  
#     """Retrieve the top k documents from the Crypto vector store based on the user question"""  
      
#     docs = vector_store.hybrid_search(question, k=k)  
#     context = "\n\n".join([doc.page_content for doc in docs])  
#     return context  

In [48]:
from langchain.agents import tool

@tool  
def retrieve(question:str, k:int=15) -> str:  
    """Retrieve the top k documents from the Crypto vector store based on the user question"""        
    docs = vector_store.hybrid_search(question, k=k)  
    context = "\n\n".join([doc.page_content for doc in docs])  
    return context  

print(retrieve("What French coffee options and bakery items do you have?"))

 
        Item Name: Pain au Chocolat
        Pain au Chocolat details:
            Price: 3.5$
            Category: Bakery
            Description: Classic French pastry filled with chocolate.
    

 
        Item Name: Black Coffee
        Black Coffee details:
            Price: 2.0$
            Category: Coffees
            Description: Strong, bold coffee with a robust flavor.
    

 
        Item Name: Cafe Breve
        Cafe Breve details:
            Price: 4.0$
            Category: Coffees
            Description: A coffee drink made with espresso and half-and-half instead of milk.
    

 
        Item Name: Mocha
        Mocha details:
            Price: 4.5$
            Category: Coffees
            Description: Delicious combination of coffee, milk and chocolate.
    

 
        Item Name: Cafe au Lait
        Cafe au Lait details:
            Price: 3.5$
            Category: Coffees
            Description: A coffee drink made with strong hot coffee and scalded milk.
  

In [49]:
tool_list = [retrieve]

In [50]:
from langchain.tools.convert_to_openai import format_tool_to_openai_function  
functions = [format_tool_to_openai_function(f) for f in tool_list]
functions

[{'name': 'retrieve',
  'description': 'retrieve(question: str, k: int = 15) -> str - Retrieve the top k documents from the Crypto vector store based on the user question',
  'parameters': {'title': 'retrieveSchemaSchema',
   'type': 'object',
   'properties': {'question': {'title': 'Question', 'type': 'string'},
    'k': {'title': 'K', 'default': 15, 'type': 'integer'}},
   'required': ['question']}}]

### `AgentExecutor` is a built-in class that provides the `run_agent` functionality

Also adds additional functionalities such as logging, error handling for tools and also the entire agent.

In [77]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import MessagesPlaceholder

model = AzureChatOpenAI(temperature=0.0,
        max_tokens=400,
        openai_api_base=openai.api_base,
        openai_api_version=openai.api_version,
        deployment_name=os.environ['GPT4_MODEL_NAME'],
        openai_api_key=openai.api_key,
        openai_api_type = openai.api_type,
        streaming=False,  
    )

model_with_tools = model.bind(functions=functions)
    
prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        """
            You are a Coffee Ordering assistant for Contoso Coffee cafe and have access to only one tool i.e. "retrieve" function. It can retrieve context based on user question from vector store that contains information about cafe's menu items. 
            Menu includes coffees, teas, bakery items, sandwiches and smoothies. As an agent, your job is to receive orders from customers and answer their inquiries about the menu items. You may need retrieve function for that to obtain information about the menu items and answer questions. 
            Feel free to skip using the tool if you don't need it to answer the question. For example, when customer is greeting or when you can find answer from conversation history. 
            Please follow these instructions when interacting with customers to generate a good and brief conversation:
            • Since you are representing Contoso, NEVER MENTION 'I am an AI language model'. Always respond in first person, not in third person. Focus solely on cafe-related queries and ordering. 
            • Keep responses concise and clear. Long responses may disengage the customers. Do not provide additional information, such as description, until explicitly asked for. Do not repeat anything until asked for, especially if you already mentioned in the conversation history.
            • If a customer is asking for recommendations, provide only top 3 most relevant recommendations. Do not provide more than 3 as it may lead to a long response.
            • Inform the customer politely if an item they request is not available in the listed menu items below.
            • If a customer likes to update or cancel the order, please help accordingly.
            • Capture any additional notes the customer may have for the menu items.
            • When listing an item in the order, mention the actual price of the item in parenthesis without multiplying it with the quantity.
            • For subtotal of the order, calculate it step-by-step: 
                ○ Total price for each ordered item = price of each ordered item * quantity of each ordered item (do this for all ordered items)
                ○ Subtotal = sum of total prices for all ordered items
                ○ Tax amount = 0.095 * Subtotal
                ○ Total = Subtotal + Tax amount
            • Do not provide the total amount of the order until asked for. If and when asked, only mention the final sum in dollars without providing the entire calculation. As mentioned earlier, remember to be brief in your responses.
            • At the end of each conversation, always obtain the customer's name, even if provided in a single word, and attach it to the order.
            • Once the customer provides their name, confirm it and state that the order can be picked up at our cafe in 15 minutes.
            • Contoso accepts only pickup orders. Payment is accepted only at the store, and if a customer suggests paying over the phone, inform them that payments are accepted solely at the store.
        Remember to always maintain a polite and professional tone.
        
        Conversaion History:
        {chat_history}
        """
                 
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "User Input: {input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [81]:
# from langchain.schema.agent import AgentFinish
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor

chain = prompt | model_with_tools | OpenAIFunctionsAgentOutputParser()

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

agent_executor = AgentExecutor(agent=agent_chain, tools=tool_list, verbose=False, memory=memory)


In [79]:
def remove_price(input_string: str) -> str:
    import re
    # Use regular expression to remove parentheses and their contents
    output_string = re.sub(r'\s*\([^)]*\)', '', input_string)
    return output_string

In [82]:
import time

print("Welcome to Contoso Coffee Inc! How can I assist you?\n")
while True:
    query = input()
    if query in ["quit", "exit"]:
        break
    start_time = time.time()
    result = agent_executor.invoke({"input": query})
    end_time = time.time()
    execution_time = end_time - start_time
    print("\nCustomer: {0}".format(query))
    print("Assistant: {0}".format(remove_price(result['output'])))
    print(f"\tTime taken to respond: {round(execution_time)} seconds")

Welcome to Contoso Coffee Inc! How can I assist you?


Customer: Hello how can you help me?
Assistant: Hello! I can assist you with placing an order from our menu which includes a variety of coffees, teas, bakery items, sandwiches, and smoothies. You can also ask me about the details of any menu item. How may I assist you today?
	Time taken to respond: 2 seconds

Customer: I am in mood for french. What french items do you have?
Assistant: We have several French-inspired items on our menu:

1. Pain au Chocolat: A classic French pastry filled with chocolate.
2. Croissant: A buttery, flaky pastry, freshly baked to golden perfection.
3. Cafe au Lait: A coffee drink made with strong hot coffee and scalded milk.

Would you like to order any of these, or is there something else you're interested in?
	Time taken to respond: 10 seconds

Customer: can I order 2 of first item and 1 of 2nd item?
Assistant: Absolutely! So, that's 2 Pain au Chocolat and 1 Croissant. Is there anything else you would 

In [83]:
from typing import List, Dict, Optional
from pydantic import BaseModel, Field, root_validator, ValidationError
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

class OrderedItem(BaseModel):
    """
    Information about each ordered item such as item name, quantity, price and additional notes provided by the customer.
    """
    item: str = Field(description="Name of the item ordered")
    quantity: int = Field(description="Quantity of that ordered item")
    price: float = Field(description="Price of that ordered item")
    item_notes: Optional[str] = Field(description="Any additional notes about that item provided by the customer")
    
class OrderInfo(BaseModel):
    """
    Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.
    Extract order details such item name, quantity, price, and any additional notes for each item.
    """
    order: List[OrderedItem] = Field(description="List of items ordered and their details")
    
class CustomerName(BaseModel):
    """
    Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.
    Extract the customer's name.
    """
    customer_name: str = Field(description="Name of the customer")
    
order_schema = convert_pydantic_to_openai_function(OrderInfo)
customer_name_schema = convert_pydantic_to_openai_function(CustomerName)

In [86]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

model = AzureChatOpenAI(temperature=0,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name="gpt-35-turbo",
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
)

extraction_functions = [convert_pydantic_to_openai_function(OrderInfo)]

order_prompt = ChatPromptTemplate.from_messages([
    ("system", """
            You are a world class algorithm for extracting information in structured formats from a restaurant order conversation between a customer and an AI assistant.
            Your task is to extract and save the pertinent details, specifically the ordered food items, quantity ordered, their prices and any additional notes requested for each ordered item included in the order. 
            Make sure to extract only those items that were ordered.
        """),
    ("user", "{input}")
])

extraction_model = model.bind(
    functions=extraction_functions,
    function_call={"name": "OrderInfo"}
)

order_extraction_chain = order_prompt | extraction_model | JsonOutputFunctionsParser()
order_result = order_extraction_chain.invoke({"input": memory.buffer_as_str})
print(json.dumps(order_result, indent=4))


{
    "order": [
        {
            "item": "Pain au Chocolat",
            "quantity": 2,
            "price": 3.5,
            "item_notes": ""
        },
        {
            "item": "Croissant",
            "quantity": 1,
            "price": 2.0,
            "item_notes": ""
        },
        {
            "item": "Cafe au Lait",
            "quantity": 1,
            "price": 3.5,
            "item_notes": "Very hot and less sweet"
        }
    ]
}


In [87]:
extraction_functions = [convert_pydantic_to_openai_function(CustomerName)]

name_prompt = ChatPromptTemplate.from_messages([
    ("system", """
            You are a world class algorithm for extracting information in structured formats from a restaurant order conversation between a customer and an AI assistant.
            Your task is to extract the customer's name included in the order.
        """),
    ("user", "{input}")
])

extraction_model = model.bind(
    functions=extraction_functions,
    function_call={"name": "CustomerName"}
)

name_extraction_chain = name_prompt | extraction_model | JsonOutputFunctionsParser()
name = name_extraction_chain.invoke({"input": memory.buffer_as_str})
customer_name = name["customer_name"]
print(f"Customer name: {customer_name}")

Customer name: Shiva


In [89]:
import pandas as pd

order_df = pd.DataFrame(order_result['order'])
order_df['item'] = order_df['item'].str.strip().str.lower()
order_df

,item,quantity,price,item_notes
0,pain au chocolat,2,3.5,
1,croissant,1,2.0,
2,cafe au lait,1,3.5,Very hot and less sweet


In [90]:
import pandas as pd  
from fuzzywuzzy import process  

menu_data = pd.read_json('../data/contoso.json') 
  
# Convert the 'Item_Name' column to lower case for standardization  
menu_data['item'] = menu_data['item'].str.lower()  
  
# Create a dictionary to store the item_ids  
item_ids = dict(zip(menu_data['item'], menu_data['id']))  
  
# Extract the item_ids for the item_names in final_order_json_list  
for item in order_result['order']:  
    item_name = item['item'].lower()  
  
    # Use fuzzy matching to find the closest matching item in the menu  
    matched_item, _ = process.extractOne(item_name, item_ids.keys())  
  
    id = item_ids.get(matched_item)  
    item['id'] = id  
  
# Print the updated final_order_json_list  
for item in order_result['order']:  
    print(item) 

{'item': 'Pain au Chocolat', 'quantity': 2, 'price': 3.5, 'item_notes': '', 'id': 'b009'}
{'item': 'Croissant', 'quantity': 1, 'price': 2.0, 'item_notes': '', 'id': 'b002'}
{'item': 'Cafe au Lait', 'quantity': 1, 'price': 3.5, 'item_notes': 'Very hot and less sweet', 'id': 'c015'}


In [91]:
def calculate_total(order):
    sum = 0
    for item in order:
        sum += item['price']*item['quantity']
    return sum

print("Your total is: {0}$".format(calculate_total(order_result['order'])))

Your total is: 12.5$


In [93]:
def print_receipt(order, customer_name, tax_percent=0.095):
    print("-----------Contoso Coffee Inc-----------\n")
    for item in order:
        print("{0} x {1} = {2}$".format(item['item'], item['quantity'], round(item['price']*item['quantity'], 2)))
        if item['item_notes'] != "":
            print("\t Notes: {0}".format(item['item_notes']))
    
    subtotal = calculate_total(order)
    tax = round(tax_percent*subtotal , 2)
    total = round(subtotal + tax, 2)
    print("\nSubtotal: {0}$".format(subtotal))
    print("Tax: {0}$".format(tax))
    print("Total: {0}$".format(total))   
    
    print(f"\nThank you for dining with us, {customer_name.title()}!")
    
print_receipt(order_result['order'], customer_name)

-----------Contoso Coffee Inc-----------

Pain au Chocolat x 2 = 7.0$
Croissant x 1 = 2.0$
Cafe au Lait x 1 = 3.5$
	 Notes: Very hot and less sweet

Subtotal: 12.5$
Tax: 1.19$
Total: 13.69$

Thank you for dining with us, Shiva!
